In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get
import re

# Pulling all links off of the search pages (up to 3000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Machine+Learning+Engineer%22&l=Boston%2C+MA&start={}"
max_results=250
Linkdf=[]

for start in range(0, max_results, 7):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

321

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
#print(websites)
len(websites)

321

In [5]:
websites1=set(websites)

In [6]:
len(websites1)

34

# Looping through websites....

In [7]:
Title=[]
Descriptions=[]
Location=[]
FullDescriptions=[]

for url in websites1:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    try:
        locations=str(location_containers.text)
    except AttributeError:
        locations = 'None Found'
    try:
        full_descriptions = str(description_containers.text)
    except AttributeError:
        full_descriptions= 'None Found'
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)
    FullDescriptions.append(full_descriptions)    

# Extracting what we want from the Descriptions Column

In [8]:
#'Location' left in for sanity check. Should be removed once code is confirmed to work
Descriptions_df = pd.DataFrame(columns = ['Title', 'Location','City', 'State', 'Zip', 'Country', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement', 'FullDescriptions'])
Country = ['US', 'USA', 'United States', 'United States of Americal']
States = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
          'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
          'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]
        Descriptions_df.at[index,"FullDescriptions"]=FullDescriptions[index]
        words = '|'.join(Country)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Country"] = temp[0]
        words = '|'.join(States)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"State"] = temp[0]
        temp = re.findall(r'\d+', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Zip"] = temp[0]  
            
        temp = re.findall(r'[\w w]+,', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"City"] = re.sub(',', '', temp[0])


In [9]:
Descriptions_df

,Title,Location,City,State,Zip,Country,Qualifications,Skills,Responsibilities,Education,Requirement,FullDescriptions
0,Senior Machine Learning Engineer,"Somerville, MA",Somerville,MA,None Found,None Found,None Found,None Found,"\nWork with software development team to manage data pipeline\nDevelop and deploy new methods around survival modeling and anomaly detection\nDesign and implement data warehouses, real-time ETL, and batch processing data to support modeling needs\nBuild and maintain internal data processing and visualization tools\n",None Found,None Found,"In this position, you will help build out data infrastructure within Data Science. The tools you build impact model development, deployment, model performance reporting, and data monitoring. You will work closely with the founders and field engineering to deliver a cutting-edge solution to our industrial clients.\n\nAs a senior data scientist on the team, you are expected to drive our machine learning strategy and overall development. We are actively researching and deploying novel machine learning methods. Part of the work includes publishing certain work in leading ML journals and presenting.\n\nResponsibilities:\n\nWork with software development team to manage data pipeline\nDevelop and deploy new methods around survival modeling and anomaly detection\nDesign and implement data warehouses, real-time ETL, and batch processing data to support modeling needs\nBuild and maintain internal data processing and visualization tools\n\nDesired skills:\n\n5+ years of data science work experience\nMasters or preferably PhD in relevant technical field\nDeep experience with Python\nExperience shipping products and features early/often\nExperience working with very large datasets, especially using Dask and Kinesis\n\nBonus Points:\n\nBackground in time series analysis, hidden Markov models, Gaussian processes, and variational inference.\nFamiliarity with node.js and both front-end/back-end JavaScript.\n\nBenefits\nWork with the founding team to solve some of the hardest problems in heavy industry (using lots of unique data!). We work with energy companies and utilities around the world to increase infrastructure reliability, reduce costs, and improve safety."
1,Machine Learning Engineer for Autonomous Systems,"Cambridge, MA",Cambridge,MA,None Found,None Found,"\nMS in Computer Science or related field, or BS with significant experience\nSoftware development experience (Python/C++)\nBroad knowledge of machine learning, particularly deep learning\nCompetency with a modern deep learning framework (e.g. PyTorch or TensorFlow)\nGood written and spoken communication skills\nApplicants selected for this position will be required to obtain and maintain a U.S. Security Clearance.","\nMS in Computer Science or related field, or BS with significant experience\nSoftware development experience (Python/C++)\nBroad knowledge of machine learning, particularly deep learning\nCompetency with a modern deep learning framework (e.g. PyTorch or TensorFlow)\nGood written and spoken communication skills\nApplicants selected for this position will be required to obtain and maintain a U.S. Security Clearance.","\nMS in Computer Science or related field, or BS with significant experience\nSoftware development experience (Python/C++)\nBroad knowledge of machine learning, particularly deep learning\nCompetency with a modern deep learning framework (e.g. PyTorch or TensorFlow)\nGood written and spoken communication skills\nApplicants selected for this position will be required to obtain and maintain a U.S. Security Clearance.",None Found,None Found,"Overview\nDraper is an independent, nonprofit research and development company headquartered in Cambridge, MA. The 1,800 employees of Draper tackle important national challenges with a promise of delivering successful and usable solutions. From military defense and space exploration to biomedical engineering, lives often depend on the solutions we provide. Our mul

# Move to csv

In [10]:
Descriptions_df.to_csv('Descriptions_df_MLE_Boston.csv')